# GARCH-Based VaR Estimation with Backtesting

In [25]:
import numpy as np, pandas as pd
from arch import arch_model
import yfinance as yf
from scipy.stats import norm

In [26]:
# Fetch historical data from S&P500
df_sp500 = yf.download("^GSPC", start='2015-01-01', end='2025-01-01', auto_adjust=True)

[*********************100%***********************]  1 of 1 completed


In [27]:
df_sp500

Price,Close,High,Low,Open,Volume
Ticker,^GSPC,^GSPC,^GSPC,^GSPC,^GSPC
Date,,,,,
2015-01-02,2058.199951,2072.360107,2046.040039,2058.899902,2708700000
2015-01-05,2020.579956,2054.439941,2017.339966,2054.439941,3799120000
2015-01-06,2002.609985,2030.250000,1992.439941,2022.150024,4460110000
2015-01-07,2025.900024,2029.609985,2005.550049,2005.550049,3805480000
2015-01-08,2062.139893,2064.080078,2030.609985,2030.609985,3934010000
...,...,...,...,...,...
2024-12-24,6040.040039,6040.100098,5981.439941,5984.629883,1757720000
2024-12-26,6037.589844,6049.750000,6007.370117,6024.970215,2904530000


In [28]:
# Eliminate MultiIndex
df_sp500.columns = df_sp500.columns.get_level_values(0)
df_sp500.columns.name = None

In [29]:
df_sp500

,Close,High,Low,Open,Volume
Date,,,,,
2015-01-02,2058.199951,2072.360107,2046.040039,2058.899902,2708700000
2015-01-05,2020.579956,2054.439941,2017.339966,2054.439941,3799120000
2015-01-06,2002.609985,2030.250000,1992.439941,2022.150024,4460110000
2015-01-07,2025.900024,2029.609985,2005.550049,2005.550049,3805480000
2015-01-08,2062.139893,2064.080078,2030.609985,2030.609985,3934010000
...,...,...,...,...,...
2024-12-24,6040.040039,6040.100098,5981.439941,5984.629883,1757720000
2024-12-26,6037.589844,6049.750000,6007.370117,6024.970215,2904530000
2024-12-27,5970.839844,6006.169922,5932.950195,6006.169922,3159610000


In [30]:
# Compute log returns
df_sp500["returns_log"] = np.log(df_sp500["Close"] / df_sp500["Close"].shift(1))

In [31]:
returns = 100 * df_sp500["returns_log"].dropna()

In [32]:
returns

Date
2015-01-05   -1.844721
2015-01-06   -0.893325
2015-01-07    1.156274
2015-01-08    1.773017
2015-01-09   -0.843932
                ...   
2024-12-24    1.098223
2024-12-26   -0.040574
2024-12-27   -1.111730
2024-12-30   -1.075967
2024-12-31   -0.429401
Name: returns_log, Length: 2515, dtype: float64

Using the past 500 days to compute the GARCH(1,1) parameters and a confidence level of $1-\alpha=0.95$, we compute the VaR and backtest it against actual losses.

In [33]:
# Rolling window
window_size = 500
confid_lvl = .95
z_score = norm.ppf(1 - confid_lvl)

# Define container to store the results
var_estimates = []
realized_losses = []

# Iterate over 500 day window
for i in range(window_size, len(returns)):
    train_data = returns[i - window_size : i]
    garch_model = arch_model(train_data, vol="Garch", p=1, q=1)
    garch_fit = garch_model.fit(disp="off")
    
    # Predict 1-day ahead volatility
    pred = garch_fit.forecast(horizon=1)
    vol_pred = np.sqrt(pred.variance.iloc[-1, 0])
    
    # Compute 1-day ahead VaR
    var_95 = z_score * vol_pred
    var_estimates.append(var_95)
    
    # Store actual return for backtesting
    realized_losses.append(returns.iloc[i])

In [36]:
# Convert to DataFrame
df_var = pd.DataFrame({
    "VaR_95": var_estimates,
    "Realized_Loss": realized_losses,
    }, index=returns.index[window_size:])

In [38]:
df_var

,VaR_95,Realized_Loss
Date,,
2016-12-28,-0.932980,-0.839164
2016-12-29,-1.128997,-0.029335
2016-12-30,-1.055854,-0.464783
2017-01-03,-1.068933,0.845077
2017-01-04,-1.175117,0.570596
...,...,...
2024-12-24,-1.516491,1.098223
2024-12-26,-1.510884,-0.040574
2024-12-27,-1.449621,-1.111730
